In [1]:
import cx_Oracle
import pandas as pd

ModuleNotFoundError: No module named 'cx_Oracle'

In [ ]:
# Load data
df = pd.read_csv("thematic_analysis.csv")

In [ ]:
# Connect to Oracle
conn = cx_Oracle.connect("your_user", "your_password", "localhost/XEPDB1")
cursor = conn.cursor()

In [ ]:
# Insert unique banks
banks = df['bank'].unique()
bank_map = {}
for bank in banks:
    cursor.execute("INSERT INTO banks (bank_name) VALUES (:1) RETURNING bank_id INTO :2",
                   [bank, cursor.var(cx_Oracle.NUMBER)])
    bank_id = cursor.fetchone()[0]
    bank_map[bank] = bank_id


    # Insert reviews
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews 
        (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, theme, source) 
        VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'), :5, :6, :7, :8)
    """, [
        bank_map[row['bank']],
        row['review'],
        int(row['rating']),
        row['date'],
        row['sentiment_label'],
        float(row['sentiment_score']),
        row['theme'],
        row['source']
    ])
    
conn.commit()
cursor.close()
conn.close()